# Data Preprocessing

#### Charlie Liou

This notebook will preprocess all of our data. The following data will be processed:
- Wikipedia "Good" articles

 1. Remove HTML brackets < >, square brackets [ ] for *n*-gram modeling
 
   - Tried to do this when scraping but there were bracket nestings. Easier to download all at once and then process


- casia_2015

 1. Prepare `casia_2015en` for *n*-gram modeling
 2. Prepare `casia_2015en` and `casia_2015cn` into a parallel corpus


- TED

 1. Prepare English portion for *n*-gram modeling?
 2. Prepare English and Chinese for sentence alignment

Function for removing HTML brackets < > and citation brackets [ ]:

Edge cases: 
    - [#]:#
    - unmatched left or right bracket

In [1]:
def remove_brackets(string):

    if "]:" in string:
        b = [[m.start(), m.end()] for m in re.finditer("(\[\w+\]:[0-9]*)|(\[\D+\]:[0-9]*)", string)] 
        for i in b:
            string = string[:i[0]] + string[i[1]:]
        remove_brackets(string)
    else:
        left = [m.start() for m in re.finditer("<|\[", string)]
        right = [m.start() for m in re.finditer(">|\]", string)]
        num = 0
        if len(left) == len(right):
            for i in range(len(left)):
                string = string[:left[i] - num] + string[right[i] + 1 - num:]
                num += right[i] - left[i] + 1
            return string
        elif len(left) > len(right): #extra left bracket
            shift, b = 0, []
            for i in range(len(right)):
                if (string[left[i + shift] - num] == "[" and string[right[i] - num] != "]") or \
                    (string[left[i + shift] - num] == "<" and string[right[i] - num] != ">"):
                    b.append(left[i] - num)
                    shift += 1
                    num += 1
            for i in b:
                string = string[:i] + " " + string[i + 1:]
            remove_brackets(string)
        else:
            shift, b = 0, []
            for i in range(len(left)): #extra right bracket
                if (string[right[i + shift] - num] == "]" and string[left[i] - num] != "[") or \
                    (string[right[i + shift] - num] == ">" and string[left[i] - num] != "<"):
                    b.append(right[i] - num)
                    shift += 1
                    num += 1
            for i in b:
                string = string[:i] + " " + string[i + 1:]        
            remove_brackets(string)
                
    return string